In [1]:
import sys, random
desaster_path = "/Users/geomando/Dropbox/github/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
import pandas as pd
import numpy as np
from desaster import entities, capitals, request, io, movement, search, rebuild

In [2]:
simulation = simpy.Environment()

write_story = True

In [3]:
def rebuild_process(simulation, human_capital, financial_capital, entity, write_story):
       
    yield simulation.process(request.inspection(simulation, human_capital, entity, write_story))
    
    if entity.residence.damage_state != 'None':
        
        money_patience = 900
        
        if entity.money_to_rebuild < entity.residence.damage_value:
            yield simulation.process(search.rebuild_money(simulation, human_capital, 
                                                            financial_capital, entity, 
                                                            money_patience, write_story))

        if entity.money_to_rebuild >= entity.residence.damage_value:
            if entity.residence.damage_state == 'Complete':
                yield simulation.process(search.permanent_housing(simulation, entity, housing_stock))
            else:
                yield simulation.process(rebuild.home(simulation, human_capital, entity, write_story))
               
   

In [4]:
def search_process(simulation, human_capital, financial_capital, entity, write_story):
    yield simulation.process(search.permanent_housing(simulation, entity, housing_stock))

In [5]:
human_cap_data = {
        "inspectors": 4,
        "insurance adjusters": 4,
        "fema processors": 4,
        "permit processors": 4,
        "contractors": 4,
        "loan processors": 4,
        "engineers": 4
                 }

human_capital = capitals.HumanCapital(simulation, human_cap_data) 

In [6]:
financial_cap_data = {"fema aid": 350000000}
# financial_cap_data = {"fema aid": 0}

financial_capital = capitals.FinancialCapital(simulation, financial_cap_data) 

In [7]:
file_path = "../inputs/household_inputs_test.csv"
households_df = pd.read_csv(file_path)

households_df.head()



,Name,Income,Savings,Insurance,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,Alfred,30000,10000,0,62 That St,Mobile Home,100,1,1,700,1920,100000,None
1,Bruce,100000,1000000,10000000,720 This Rd,Single Family Dwelling,100000,6,5,5000,1920,10000000,Slight
2,Selena,10000,100,0,1001 Other Ave,Multi Family Dwelling,10,0,1,250,1960,10000,Complete
3,Fish,50000,1000,550000,26000 Out There Lane,Single Family Dwelling,2000,4,2,2000,2010,800000,Moderate


In [8]:
household = {}
for i in households_df.index:
    household[i] = entities.Household(simulation, households_df.iloc[i], write_story)

In [9]:
file_path = "../inputs/housing_stock_test.csv"
housing_stock_df = pd.read_csv(file_path)

housing_stock = capitals.setHousingStock(simulation, housing_stock_df)

housing_stock_df.head()

,Address,Occupancy,Cost,Bedrooms,Bathrooms,Area,Year Built,Value,Damage State
0,99 Growl Ave,Mobile Home,100,1,1,700,1920,100000,None
1,1100 Woof Rd,Single Family Dwelling,100000,6,5,5000,1920,10000000,Slight
2,32 Meow Blvd,Multi Family Dwelling,10,0,1,250,1960,10000,None
3,3400 Moo Lane,Single Family Dwelling,2000,4,2,2000,2010,800000,Slight


In [10]:
for i in range(len(household)):
    simulation.process(rebuild_process(simulation, human_capital, financial_capital, household[i], write_story))

# for i in range(len(household)):
#     simulation.process(search_process(simulation, human_capital, financial_capital, household[i], write_story))

In [11]:
simulation.run()

In [12]:
household[0].story

['Alfred lives in a 1 bedroom Mobile Home at 62 That St. ',
 "Alfred's house was inspected 10.0 days after the event and suffered $0.0 of damage."]

In [13]:
household[1].story

['Bruce lives in a 6 bedroom Single Family Dwelling at 720 This Rd. ',
 "Bruce's house was inspected 10.0 days after the event and suffered $200000.0 of damage.",
 'Bruce received an engineering assessment 35.0 days after the event. ',
 'Bruce received permit approval 70.0 days after the event. ',
 'The house was rebuilt 72.0 days after the event, taking 2.0 days to rebuild. ']

In [14]:
household[2].story

['Selena lives in a 0 bedroom Multi Family Dwelling at 1001 Other Ave. ',
 "Selena's house was inspected 10.0 days after the event and suffered $10000.0 of damage.",
 'Selena submitted a request to FEMA 10.0 days after the event. ',
 'Selena received $10000.0 from FEMA 30.0 days after the event. ',
 'It took Selena 20.0 days to exhaust the search for money. ',
 'Selena started searching for a Multi Family Dwelling with a value under $10000 on day 30.0. ',
 'On day 30.0, Selena found a Multi Family Dwelling at 32 Meow Blvd with a value of $10000. ']

In [15]:
household[3].story

['Fish lives in a 4 bedroom Single Family Dwelling at 26000 Out There Lane. ',
 "Fish's house was inspected 10.0 days after the event and suffered $80000.0 of damage.",
 'Fish submitted an insurance claim 10.0 days after the event. ',
 'Fish received a $80000.0 insurance payout 25.0 days after the event. ',
 'It took Fish 15.0 days to exhaust the search for money. ',
 'Fish received an engineering assessment 50.0 days after the event. ',
 'Fish received permit approval 85.0 days after the event. ',
 'The house was rebuilt 115.0 days after the event, taking 30.0 days to rebuild. ']

In [16]:
household[1].money_search_start

0.0